<a href="https://colab.research.google.com/github/GabAlvarenga/CursoProgramacaoAvancadaENCE/blob/main/SearshGrid_RandomGrid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import time
# Importando dataset : Breast_Cancer
import sklearn.datasets
from sklearn.datasets import load_breast_cancer
data, target = load_breast_cancer(return_X_y=True, as_frame=True)

# Train, Test, Split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.10, 
                                                    shuffle=True)

# Criando modelo 1: Arvore de decisão
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier() 

# Criando Grid de Pârametros
Grid = {"max_depth":[2,3,4,6,8,10,12,15,20],
        "min_samples_leaf":[1,2,4,6,8,10,20,30],
        'min_samples_split':[1.0,2,3,4,5,6,8,10]}

# Searsh Grid###############################################################################################################
start_gs = time.time()
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(tree, param_grid=Grid, 
                  scoring="accuracy",
                  n_jobs=-1,
                  cv=10,verbose=1)

gs.fit(X_train, y_train)

end_gs = time.time()
tempo_gs = end_gs - start_gs


#Random Grid#################################################################################################################
start_rs=time.time()
from sklearn.model_selection import RandomizedSearchCV

rs = RandomizedSearchCV(tree, param_distributions=Grid,
                        n_iter=8, scoring="accuracy",
                        n_jobs=-1, 
                        cv=10,verbose=1)

rs.fit(X_train, y_train)
end_rs = time.time()
tempo_rs = end_rs - start_rs


##############################################################################################################################
# Avaliação dos métodos
gs_param = gs.best_estimator_.get_params(deep=True)
rs_param = rs.best_estimator_.get_params(deep=True)
Comparaçao_metodos = pd.DataFrame({"max_depth":[gs_param["max_depth"], rs_param["max_depth"]],
                                    "min_samples_leaf":[gs_param["min_samples_leaf"], rs_param["min_samples_leaf"]],
                                    'min_samples_split':[gs_param['min_samples_split'], rs_param['min_samples_split']],
                                   "Accuracy treino ":[gs.best_estimator_.score(X_train,y_train), rs.best_estimator_.score(X_train,y_train)],
                                   "Accuracy teste":[gs.best_estimator_.score(X_test,y_test), rs.best_estimator_.score(X_test,y_test)],
                                   "Tempo de Execução (Segundos)":["{:.2f}".format(tempo_gs,2),"{:.2f}".format(tempo_rs,2)]  
},index=["GridSearchCV","RandomizedSearchCV"])
Comparaçao_metodos

Fitting 10 folds for each of 576 candidates, totalling 5760 fits
Fitting 10 folds for each of 8 candidates, totalling 80 fits


,max_depth,min_samples_leaf,min_samples_split,Accuracy treino,Accuracy teste,Tempo de Execução (Segundos)
GridSearchCV,8,1,6,0.992188,0.982456,52.50
RandomizedSearchCV,12,6,3,0.978516,0.982456,0.76


In [ ]:
rs.best_estimator_.score(X_test,y_test)

0.9122807017543859

In [ ]:
 pd.DataFrame(gs.cv_results_)[["param_max_depth","param_min_samples_leaf","param_min_samples_split","mean_test_score","rank_test_score"]].sort_values("rank_test_score")

,param_max_depth,param_min_samples_leaf,param_min_samples_split,mean_test_score,rank_test_score
209,6,4,2,0.939453,1
342,10,4,8,0.939453,1
340,10,4,5,0.939453,1
533,20,4,6,0.937500,4
277,8,4,6,0.937500,4
...,...,...,...,...,...
240,6,20,1,0.894531,505
248,6,30,1,0.894531,505
448,15,1,1,0.894531,505
208,6,4,1,0.894531,505


In [ ]:
pd.DataFrame(rs.cv_results_)[["param_max_depth","param_min_samples_leaf","param_min_samples_split","mean_test_score","rank_test_score"]].sort_values("rank_test_score")

,param_max_depth,param_min_samples_leaf,param_min_samples_split,mean_test_score,rank_test_score
6,4,6,10,0.935547,1
9,4,2,5,0.933594,2
1,4,1,10,0.931641,3
2,15,1,10,0.927734,4
0,2,1,3,0.925781,5
4,2,2,5,0.925781,5
8,8,2,5,0.921875,7
7,15,20,4,0.919922,8
5,8,20,3,0.917969,9
3,15,1,1,0.876953,10
